# Data Preparation Notebook
This notebook transforms the source dataset into a single dataset that can be used to train machine learning models.

First, we'll perform data munging on our data to get it into an appropriate shape.

After that is complete, we'll use the Azure ML SDK to upload this data to Azure as a registered dataset.

### Overview
I am using the [NHL Game Data dataset](https://www.kaggle.com/datasets/martinellis/nhl-game-data) from Kaggle.com compiled by Martin Ellis which contains a series of CSV files containing NHL hockey game and player data from 2001 to 2020.

![Kaggle Dataset](screens/Kaggle.png)

The dataset is stored in multiple CSV files structured like a relational database. In fact, the author of the dataset was so kind as to provide this schema diagram of the dataset below:

![Schema Diagram](data/table_relationships.JPG)

## Data Munging
Our source dataset is in many different files. We need to join various files together and aggregate down to game-level information before we can use it for training.

In [5]:
# Common Imports
import pandas as pd
from azureml.core import Workspace, Dataset

In [6]:
# Team stats is the most efficient way of getting pre-aggregated penalty data. Let's load it up and reduce it down to the total penalties per game
df_team_stats = pd.read_csv('data/game_teams_stats.csv')[['game_id', 'pim']].rename(columns={'pim': 'penaltyMinutes'}).groupby('game_id').agg('sum')
df_team_stats.head()

,penaltyMinutes
game_id,
2000020001,42.0
2000020002,32.0
2000020003,45.0
2000020004,24.0
2000020005,52.0


In [7]:
# Load Game Data
df_games = pd.read_csv('data/game.csv')
df_games.head()

,game_id,season,type,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2016020045,20162017,R,2016-10-19T00:30:00Z,4,16,4,7,home win REG,right,United Center,/api/v1/venues/null,America/Chicago,-5,CDT
1,2017020812,20172018,R,2018-02-07T00:00:00Z,24,7,4,3,away win OT,left,KeyBank Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2015020314,20152016,R,2015-11-24T01:00:00Z,21,52,4,1,away win REG,right,MTS Centre,/api/v1/venues/null,America/Winnipeg,-5,CDT
3,2015020849,20152016,R,2016-02-17T00:00:00Z,52,12,1,2,home win REG,right,PNC Arena,/api/v1/venues/null,America/New_York,-4,EDT
4,2017020586,20172018,R,2017-12-30T03:00:00Z,20,24,1,2,home win REG,left,Honda Center,/api/v1/venues/null,America/Los_Angeles,-7,PDT


In [8]:
# Aggregate everything together
df_detailed_pens = pd.merge(left=df_team_stats, right=df_games, how='left', left_on='game_id', right_on='game_id')

# Reduce down to core columns useful for predicting game penalties ahead of time
df_detailed_pens = df_detailed_pens[['game_id', 'penaltyMinutes', 'season', 'type', 'away_team_id', 'home_team_id']]

df_detailed_pens.head()

,game_id,penaltyMinutes,season,type,away_team_id,home_team_id
0,2000020001,42.0,20002001,R,21,25
1,2000020002,32.0,20002001,R,9,6
2,2000020003,45.0,20002001,R,16,7
3,2000020004,24.0,20002001,R,23,4
4,2000020005,52.0,20002001,R,17,20


In [9]:
# For fun, let's look at aggregate penalty minutes
df_detailed_pens.penaltyMinutes.describe()

count    26305.000000
mean        26.806615
std         18.972391
min          0.000000
25%         16.000000
50%         22.000000
75%         32.000000
max        419.000000
Name: penaltyMinutes, dtype: float64

In [10]:
# We need to know a bit more about who these teams are. Load a lookup table
df_teams = pd.read_csv('data/team_info.csv')[['team_id', 'teamName']]
df_teams.head()

,team_id,teamName
0,1,Devils
1,4,Flyers
2,26,Kings
3,14,Lightning
4,6,Bruins


In [11]:
# Now get the home and away teams into the detailed penalties dataframe
df_detailed_pens = pd.merge(left=df_detailed_pens, right=df_teams, how='inner', left_on='home_team_id', right_on='team_id')
df_detailed_pens = df_detailed_pens.rename(columns={'teamName': 'homeTeam'})
df_detailed_pens = df_detailed_pens.drop(['home_team_id','team_id'], axis=1)

df_detailed_pens = pd.merge(left=df_detailed_pens, right=df_teams, how='inner', left_on='away_team_id', right_on='team_id')
df_detailed_pens = df_detailed_pens.rename(columns={'teamName': 'awayTeam'})
df_detailed_pens = df_detailed_pens.drop(['away_team_id','team_id'], axis=1)

df_detailed_pens.head()

,game_id,penaltyMinutes,season,type,homeTeam,awayTeam
0,2000020001,42.0,20002001,R,Stars,Avalanche
1,2000021024,20.0,20002001,R,Stars,Avalanche
2,2001020550,12.0,20012002,R,Stars,Avalanche
3,2001021160,16.0,20012002,R,Stars,Avalanche
4,2002020249,20.0,20022003,R,Stars,Avalanche


In [12]:
# Historical information is helpful, but does become outdated. Let's look at 2015 onwards, then drop our identifiers that might throw off machine learning
df_training = df_detailed_pens[df_detailed_pens['season'] >= 20152016].drop(columns=['game_id', 'season'])

print(df_training.shape)

df_training.head()

(9142, 4)


,penaltyMinutes,type,homeTeam,awayTeam
28,12.0,R,Stars,Avalanche
29,29.0,R,Stars,Avalanche
30,18.0,R,Stars,Avalanche
31,24.0,R,Stars,Avalanche
32,4.0,R,Stars,Avalanche


## Uploading to Azure
Now that we have data properly aggregated and in a state ready to feed to a machine learning model, let's upload it to Azure as a dataset

In [13]:
# Connect to the workspace
ws = Workspace.from_config() # config.json is needed for this

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: DataScience
Azure region: northcentralus
Resource group: datascience


In [14]:
datastore = ws.get_default_datastore()

ds = Dataset.Tabular.register_pandas_dataframe(dataframe=df_training, name='NHL-Penalties-2020', description='A breakdown of penalty minutes per game matchup', target=datastore)
ds

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e82669e2-8595-4c8d-a2aa-ae7982a3e7f1/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/e82669e2-8595-4c8d-a2aa-ae7982a3e7f1/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "3eec7f38-eba2-49e7-abf7-d18ef0ad00ee",
    "name": "NHL-Penalties-2020",
    "version": 4,
    "description": "A breakdown of penalty minutes per game matchup",
    "workspace": "Workspace.create(name='DataScience', subscription_id='efba8785-116c-4443-9a05-764c75c7bb0d', resource_group='datascience')"
  }
}